# Scripts Wrapped

In [5]:
# This Pbdate_rangeython 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/microbes-dataset/microbes.csv
/kaggle/input/bank-market/bank_test_label.csv
/kaggle/input/bank-market/bank_train_label.csv


In [6]:
!pip install openml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 3.1 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 5.6 MB/s eta 0:00:00
  Created wheel for openml: filename=openml-0.14.2-py3-none-any.whl size=158701 sha256=75c77d26a73028e89d89e87c13d97ebc4f2c2f39509d9470a63def2434a8f413
  Stored in directory: /root/.cache/pip/wheels/2e/4e/af/5e721761d86375dbca82e63cc2470019e97815bc39f11451ea
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11716 sha256=16d94ecbf3ac0a1e70c56a86ba194e4b69fbde74a8abe757028612a043ab8e45
  Stored in directory: /root/.cache/pip/wheels/5d/2a/9c/3895d9617f8f49a0883ba686326d598e78a1c2f54fe3cae86d
Successfully built openml liac-arff


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn import preprocessing
import openml
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report
import time

In [8]:
def get_dataset_by_id(id):
  dataset = openml.datasets.get_dataset(id)

  # Get the data itself as a dataframe (or otherwise)
  df, y, one, columns = dataset.get_data(dataset_format="dataframe")
  return [df, columns]

In [9]:
def split_data(df, ratio):


  X = df.iloc[:, :-1]
  standardize = StandardScaler()
  X = np.array(standardize.fit_transform(X))
  Y = df.iloc[:, -1]
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = ratio)
  return [X_train, X_test, Y_train, Y_test]

In [10]:
def print_shapes(X_train, X_test, Y_train, Y_test):

  print("Shape of training data", X_train.shape)
  print("Shape of training labels", Y_train.shape)
  print("Shape of test data", X_test.shape)
  print("Shape of test labels", Y_test.shape)

In [11]:
def perform_grid_search(X_train, Y_train):
  # Initialize the Random Forest classifier
  rf_classifier = RandomForestClassifier(random_state=42)

  # Define the parameter grid
  param_grid = {
      "n_estimators": [100, 200, 250, 300, 350, 400, 500],
      "criterion": ["gini", "entropy"],
      "max_features": ['sqrt', 'log2', None],
      "max_depth": [3, 4, 6, 10, 15, 20, 25, 30, 35, 40],
      "min_samples_split": [3]
  }

  # Perform Grid Search with 5-fold cross-validation
  grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
  grid_search.fit(X_train, Y_train)
  return grid_search


In [12]:
def get_best_params(grid_search, X_test, Y_test):

  print("Best Parameters:", grid_search.best_params_)
  print()

  # Predict using the best estimator found by Grid Search
  y_pred = grid_search.predict(X_test)

  # Print classification report and accuracy score on the test set
  print("Classification Report:")

  print(classification_report(Y_test, y_pred))
  print("Accuracy Score:", accuracy_score(Y_test, y_pred))

In [13]:
def train_model(X_train, Y_train, best_params, X_test, Y_test):
  # Construct the Random Forest model with the best parameters
  # best_params = {
  #     'criterion': 'entropy',
  #     'max_depth': 10,
  #     'max_features': 'sqrt',  # Choose one value from ['sqrt', 'log2', None']
  #     'min_samples_split': 3,
  #     'n_estimators': 200
  # }

    rf_classifier = RandomForestClassifier(
      criterion=best_params['criterion'],
      max_depth=best_params['max_depth'],
      max_features=best_params['max_features'],
      min_samples_split=best_params['min_samples_split'],
      n_estimators=best_params['n_estimators'],
      random_state=42
    )

  # Measure the training time
    start_time = time.time()
    rf_classifier.fit(X_train, Y_train)
    end_time = time.time()
    
    start = time.time()
    y_pred = rf_classifier.predict(X_test)


  # Predict on the test set

  # Calculate evaluation metrics
    accuracy = accuracy_score(Y_test, y_pred)
    end = time.time()
    f1 = f1_score(Y_test, y_pred, average='weighted')
    recall = recall_score(Y_test, y_pred, average='weighted')
    precision = precision_score(Y_test, y_pred, average='weighted')

    training_time = end_time - start_time
    inference_time = end - start
  # Print the evaluation metrics
    print(f"Training Time: {training_time:.2f} seconds")
    print(f"Inference Time: {inference_time:.2f} seconds")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"Precision: {precision:.2f}")

  # Print the detailed classification report
    print("\nClassification Report:")
    print(classification_report(Y_test, y_pred))
    return rf_classifier


In [14]:
# function name -  function - parameters

# get_dataset_by_id - gets dataset from openml by id  - id
# split_data       - splits dataset into training and test dataset - dataframe, ratio
# print_shapes    - print shapes of X_train, Y_train, X_test, Y_test
#perform_grid_search - grid search on training dataset - X_train, Y_train
#get grid

# Train on Our Preprocess

In [15]:
# df_train = pd.read_csv("/kaggle/input/bank-market/bank_train_label.csv")
# df_train

In [16]:
# df_test = pd.read_csv("/kaggle/input/bank-market/bank_test_label.csv")
# df_test

In [13]:
# df_bank = pd.concat([df_train, df_test])
# df_bank

In [14]:

# X_train, X_test, Y_train, Y_test = split_data(df_bank, 0.3)
# print_shapes(X_train, X_test, Y_train, Y_test)


In [15]:
# grid_search = perform_grid_search(X_train, Y_train)

In [16]:
# get_best_params(grid_search, X_test, Y_test)

In [17]:
# rf_1 = train_model(X_train, Y_train, grid_search.best_params_, X_test, Y_test)

In [18]:
# import joblib
# import os

# # Save the model
# joblib.dump(rf_1, 'model.pkl')

# # Get the file size
# model_size = os.path.getsize('model.pkl') / (1024 * 1024)  # Size in MB
# print(f"Model Size: {model_size:.2f} MB")

In [17]:
df, columns = get_dataset_by_id(44126)

/tmp/ipykernel_33/2299138871.py:2: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = openml.datasets.get_dataset(id)


In [18]:
df

,V1,V6,V10,V12,V13,V14,V15,Class
0,52,51.0,14,935.0,2,-1.0,0,1
1,53,232.0,29,32.0,1,-1.0,0,1
2,50,525.0,18,177.0,2,-1.0,0,1
3,32,648.0,21,166.0,1,-1.0,0,1
4,58,2348.0,9,226.0,1,-1.0,0,1
...,...,...,...,...,...,...,...,...
10573,73,2850.0,17,300.0,1,40.0,8,2
10574,25,505.0,17,386.0,2,-1.0,0,2
10575,51,825.0,17,977.0,3,-1.0,0,2
10576,71,1729.0,17,456.0,2,-1.0,0,2


In [21]:
#Two classes with equal values

df["Class"].value_counts()

Class
1    5289
2    5289
Name: count, dtype: int64

In [22]:
df_train = df.iloc[:10000, :]
df_train

,V1,V6,V10,V12,V13,V14,V15,Class
0,52,51.0,14,935.0,2,-1.0,0,1
1,53,232.0,29,32.0,1,-1.0,0,1
2,50,525.0,18,177.0,2,-1.0,0,1
3,32,648.0,21,166.0,1,-1.0,0,1
4,58,2348.0,9,226.0,1,-1.0,0,1
...,...,...,...,...,...,...,...,...
9995,32,3255.0,29,167.0,2,-1.0,0,2
9996,32,64.0,30,598.0,4,105.0,5,2
9997,48,1544.0,30,263.0,1,450.0,2,2
9998,41,985.0,30,274.0,2,391.0,11,2


In [23]:
df_test = df.iloc[10001:43307, :]
df_test

,V1,V6,V10,V12,V13,V14,V15,Class
10001,21,2488.0,30,258.0,6,169.0,3,2
10002,74,29080.0,30,301.0,5,93.0,4,2
10003,32,12569.0,1,561.0,2,295.0,2,2
10004,63,2896.0,1,304.0,1,119.0,2,2
10005,53,36.0,2,25.0,1,417.0,1,2
...,...,...,...,...,...,...,...,...
10573,73,2850.0,17,300.0,1,40.0,8,2
10574,25,505.0,17,386.0,2,-1.0,0,2
10575,51,825.0,17,977.0,3,-1.0,0,2
10576,71,1729.0,17,456.0,2,-1.0,0,2


In [24]:
X_trn = np.array(df_train.iloc[:, :-1])
Y_trn = np.array(df_train.iloc[:, -1])
X_tst = np.array(df_test.iloc[:, :-1])
Y_tst = np.array(df_test.iloc[:, -1])
print_shapes(X_trn, X_tst, Y_trn, Y_tst)

Shape of training data (10000, 7)
Shape of training labels (10000,)
Shape of test data (577, 7)
Shape of test labels (577,)


In [16]:
# X_trn, X_tst, Y_trn, Y_tst = split_data(df, 0.3)

In [25]:
# #split the dataset by the ratio 0.8:0.2 ratio

# X_trn, X_tst, Y_trn, Y_tst = split_data(df, 0.6)
# print_shapes(X_trn, X_tst, Y_trn, Y_tst)

In [26]:
print(type(X_trn))

<class 'numpy.ndarray'>


In [17]:
grid_search_2 = perform_grid_search(X_trn, Y_trn)

Fitting 5 folds for each of 420 candidates, totalling 2100 fits


KeyboardInterrupt: 

In [28]:
get_best_params(grid_search_2, X_tst, Y_tst)

In [29]:
# rf_2 = train_model(X_trn, Y_trn, grid_search_2.best_params_, X_tst, Y_tst)

In [30]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_trn = scaler.fit_transform(X_trn)
# X_tst = scaler.fit_transform(X_tst)


In [25]:


rf_classifier = RandomForestClassifier(
  criterion='gini',
  max_depth=20,
  max_features='sqrt',
  min_samples_split=2,
  n_estimators=500,
    bootstrap = True,
    min_samples_leaf = 10,
    random_state = 42,
)

# Measure the training time
start_time = time.time()
rf_classifier.fit(X_trn, Y_trn)
end_time = time.time()

start = time.time()
y_pred = rf_classifier.predict(X_tst)


# Predict on the test set

# Calculate evaluation metrics
accuracy = accuracy_score(Y_tst, y_pred)
end = time.time()
f1 = f1_score(Y_tst, y_pred, average='weighted')
recall = recall_score(Y_tst, y_pred, average='weighted')
precision = precision_score(Y_tst, y_pred, average='weighted')

training_time = end_time - start_time
inference_time = end - start
# Print the evaluation metrics
print(f"Training Time: {training_time:.2f} seconds")
print(f"Inference Time: {inference_time:.2f} seconds")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Precision: {precision:.2f}")

# Print the detailed classification report
print("\nClassification Report:")
print(classification_report(Y_tst, y_pred))
# return rf_classifier


Training Time: 6.00 seconds
Inference Time: 0.08 seconds
Accuracy: 0.8440207972270364
F1 Score: 0.92
Recall: 0.84
Precision: 1.00

Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       1.00      0.84      0.92       577

    accuracy                           0.84       577
   macro avg       0.50      0.42      0.46       577
weighted avg       1.00      0.84      0.92       577



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarni

In [32]:
# import joblib
# import os

# # Save the model
# joblib.dump(rf_2, 'model.pkl')

# # Get the file size
# model_size = os.path.getsize('model.pkl') / (1024 * 1024)  # Size in MB
# print(f"Model Size: {model_size:.2f} MB")